In [1]:
# magic lines that avoid re-start 
%load_ext autoreload
%autoreload 2
import pybamm as pb;import pandas as pd   ;import numpy as np;
import os; import seaborn as sns
import matplotlib.pyplot as plt;import os;import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;
import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import openpyxl; import json
import traceback
import multiprocessing
import scipy.optimize
import timeit
import random;import time, signal
fs=17;

font = {'family' : 'DejaVu Sans',  'size'   : fs}
mpl.rc('font', **font)

In [2]:
# define global index and dict for all experiment data - prepare for read!
Exp_Path = [
    "Expt 1 - Si-based Degradation/",
    "Expt 2,2 - C-based Degradation 2/",
    "Expt 3 - Cathode Degradation and Li-Plating/",
    "Expt 4 - Drive Cycle Aging (Control)/",
    "Expt 5 - Standard Cycle Aging (Control)/",]
Exp_head = [
    "Expt 1",
    "Expt 2,2",
    "Expt 3",
    "Expt 4",
    "Expt 5",]
Exp_1_Cell = ["A","B","J","D","E","F","K","L","M"];
Exp_1_Temp = {
    "A":"10","B":"10","J":"10",
    "D":"25","E":"25","F":"25",
    "K":"40","L":"40","M":"40",}
Temp_Cell_Exp_1 = {
    "10":["A","B","J"],
    "25":["D","E","F"],
    "40":["K","L","M"],}
Exp_2_Cell = ["A","B","C","D","E","F"];
Exp_2_Temp = {
    "A":"10","B":"10",
    "C":"25","D":"25",
    "E":"40","F":"40",}
Temp_Cell_Exp_2 = {
    "10":["A","B"],
    "25":["C","D"],
    "40":["E","F"],}
Exp_3_Cell = ["A","B","C","D","E","F","G","H","I"];
Exp_3_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25","F":"25",
    "G":"40","H":"40","I":"40"}
Temp_Cell_Exp_3 = {
    "10":["A","B","C"],
    "25":["D","E","F"],
    "40":["G","H","I"],}
Exp_4_Cell = ["A","B","C","D","E","F","G","H"];
Exp_4_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25",
    "F":"40","G":"40","H":"40",}
Temp_Cell_Exp_4 = {
    "10":["A","B","C"],
    "25":["D","E",],
    "40":["F","G","H"],}
Exp_5_Cell = ["A","B","C","D","E","F","G","H"];
Exp_5_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25",
    "F":"40","G":"40","H":"40",}
Temp_Cell_Exp_5 = {
    "10":["A","B","C"],
    "25":["D","E",],
    "40":["F","G","H"],}
Exp_All_Cell  = [Exp_1_Cell,Exp_2_Cell,Exp_3_Cell,Exp_4_Cell,Exp_5_Cell]
Exp_Temp_Cell = [Exp_1_Temp,Exp_2_Temp,Exp_3_Temp,Exp_4_Temp,Exp_5_Temp]
Temp_Cell_Exp_All = [Temp_Cell_Exp_1,Temp_Cell_Exp_2,Temp_Cell_Exp_3,Temp_Cell_Exp_4,Temp_Cell_Exp_5]
Mark_Cell_All = [
    {
    "A":"o","B":">","J":"v",
    "D":"o","E":">","F":"v",
    "K":"o","L":">","M":"v",},
    {
    "A":"10","B":"10",
    "C":"25","D":"25",
    "E":"40","F":"40",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">","F":"v",
    "G":"o","H":">","I":"v",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">",
    "F":"o","G":">","H":"v",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">",
    "F":"o","G":">","H":"v",}]
Color_Cell_All = [
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"J":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],"F":[0, 0, 0,0.7],
    "K":[1,0,0,0.4],"L":[1,0,0,0.4],"M":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"C":[0, 0, 0,0.7],
    "E":[1,0,0,0.4],"F":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],"F":[0, 0, 0,0.7],
    "G":[1,0,0,0.4],"H":[1,0,0,0.4],"I":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],
    "F":[1,0,0,0.4],"G":[1,0,0,0.4],"H":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],
    "F":[1,0,0,0.4],"G":[1,0,0,0.4],"H":[1,0,0,0.4],}]


In [30]:
# ready experiment data
import sys  
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip/Rio_Code/Fun_P2"))
sys.path.append(str_path_1) 
from Fun_P2 import * # directly do this for HPC
# Load Niall's data
Path_NiallThrCap = os.path.expanduser(
    "~/EnvPBGEM_Linux/SimSave/Cal_ThroughputCap/Expt 2,2 - C-based Degradation 2/") 
str_RPT = "Performance Checks"; str_AGE = "Degradation Cycling"; str_saveFig = "Figures"

In [ ]:
# Function: read the data file and get the pandas data frame
def Read_Charge_Throuput(Output_dict,mpt_file,str_appx,str,str_saveFig,dpi): # str can be str_AGE or str_RPT
    # step-1: get the RPT name - input: filename
    degC_index = mpt_file.find('degC')
    mpt_index = mpt_file.find(str_appx)
    if degC_index != -1 and mpt_index != -1:
        extracted_part = mpt_file[degC_index -2  :  mpt_index]
        # print("Extracted Part:", extracted_part)
    else:
        print("Either 'degC' or '.mpt' not found in the filename.")
    # step-2: read data file
    colsToKeep = [
        "time/s", "Ecell/V", "I/mA", "(Q-Qo)/mA.h", 
        "Temperature/ｰC", "Q discharge/mA.h", "Ns changes"]
    data = pd.read_csv(
        Path_NiallThrCap + str + "/" + mpt_file,
        encoding = "shift-jis", sep = "\t", usecols = colsToKeep)
    # step-3: plot and save 
    fig, axs = plt.subplots(figsize=(6,5),tight_layout=True)
    axs.plot(data["time/s"]/3600,data["Ecell/V"])
    axs.set_ylabel("Voltage / V")
    axs.set_xlabel("Time / h")
    axs.set_title(extracted_part,fontsize=fs)
    plt.savefig(Path_NiallThrCap + str_saveFig + f"/Voltage of {extracted_part}.png", dpi=dpi)
    plt.close()
    # step-4: get output dataframe
    Start_time = data["time/s"].iloc[0]/3600
    End_time   = data["time/s"].iloc[-1]/3600
    Charge_throughput = np.trapz(abs(data['I/mA'])/1E3, data["time/s"]/3600) # in A.h
    print(extracted_part,Start_time,End_time,Charge_throughput)
    Output_dict["File_name"].append(extracted_part)
    Output_dict["Start_time [h]"].append(Start_time)
    Output_dict["End_time [h]"].append(End_time)
    Output_dict["Charge_throughput [A.h]"].append(Charge_throughput)
    print(f"Successfully process {extracted_part}")
    return Output_dict


#### Now process the RPT

In [ ]:
all_files = os.listdir(Path_NiallThrCap + str_RPT)
mpt_files = [file for file in all_files if file.endswith('.mpt')]
Output_dict = {}
Output_dict["File_name"] = []
Output_dict["Start_time [h]"] = []
Output_dict["End_time [h]"] = []
Output_dict["Charge_throughput [A.h]"] = []
for mpt_file in mpt_files:
    Output_dict = Read_Charge_Throuput(Output_dict,mpt_file,".mpt",str_RPT,str_saveFig,dpi=300)

In [56]:
# Create a DataFrame from the dictionary
df_RPT = pd.DataFrame(Output_dict)
df_RPT

,File_name,Start_time [h],End_time [h],Charge_throughput [A.h]
0,25degC - cell C - RPT4_02_MB_CA4,28.753736,41.053962,8.320103
1,10degC - cell A - BoL - RPT0_short_02_MB_CB4,30.116396,42.002641,8.627140
2,25degC - cell D - RPT3_01_MB_CA5,0.000000,28.926799,11.701298
3,25degC - cell D - RPT6_04_MB_CA5,76.199805,93.839259,9.049132
4,25degC - cell D - RPT8_04_MB_CA5,75.941134,93.554371,8.901604
...,...,...,...,...
289,10degC - cell B - RPT9_02_MB_CB5,27.727042,39.504122,7.734937
290,40degC - cell E - RPT12_02_MB_CF4,27.602702,39.654108,7.794994
291,25degC - cell D - BoL - RPT0_short_02_MB_CA5,30.060644,41.941608,8.460551
292,25degC - cell C - RPT6_04_MB_CA4,76.130150,93.735273,9.084391


In [188]:
# Function to extract values
def extract_values(file_name):
    index_s = file_name.find("RPT")
    index_e = file_name.find("_MB_")
    index_c = file_name.find("cell")
    
    RPT = file_name[index_s + 3:index_e]
    if file_name[index_s + 4] in [str(i) for i in range(10)]:
        RPT_No = int( file_name[index_s + 3 : index_s + 5]  )
    else:
        RPT_No = int( file_name[index_s + 3]  )
    T_age = file_name[0:2]
    str_cell = file_name[index_c + 5]
    return pd.Series({'RPT': RPT, "RPT_No":RPT_No,'T_age': T_age, 'cell': str_cell})

# Apply the function to each row and create new columns
df_RPT[['RPT', "RPT_No",'T_age', 'cell']] = df_RPT['File_name'].apply(extract_values)
df_RPT_sorted = df_RPT.sort_values(by=['cell', "RPT_No"])
df_RPT_sorted = df_RPT_sorted.reset_index(drop=True)
df_RPT_sorted


,File_name,Start_time [h],End_time [h],Charge_throughput [A.h],RPT,T_age,cell,RPT_No
0,10degC - cell A - BoL - RPT0_short_02_MB_CB4,30.116396,42.002641,8.627140,0_short_02,10,A,0
1,10degC - cell A - BoL - RPT0_long_04_MB_CB4,78.242328,96.138401,9.585447,0_long_04,10,A,0
2,10degC - cell A - BoL - RPT0_long_03_MB_CB4,42.634081,78.242325,5.675480,0_long_03,10,A,0
3,10degC - cell A - BoL - RPT0_short_03_MB_CB4,42.002661,52.284793,5.658056,0_short_03,10,A,0
4,10degC - cell A - BoL - RPT0_long_02_MB_CB4,30.165975,42.634078,8.770421,0_long_02,10,A,0
...,...,...,...,...,...,...,...,...
289,40degC - cell F - RPT11_03_MB_CF5,39.267307,49.460883,5.079796,11_03,40,F,11
290,40degC - cell F - RPT12_03_MB_CF5,39.462416,74.808053,5.177661,12_03,40,F,12
291,40degC - cell F - RPT12_02_MB_CF5,27.454973,39.462413,7.717736,12_02,40,F,12
292,40degC - cell F - RPT12_04_MB_CF5,74.808056,92.192895,8.612296,12_04,40,F,12


In [191]:
df_RPT_summed =  df_RPT_sorted.groupby(['RPT_No', 'cell'])['Charge_throughput [A.h]'].sum().reset_index()
# df_summed = df_RPT_sorted.groupby(['RPT_No', 'cell'])['Charge_throughput [A.h]'].sum().reset_index()
df_RPT_summed = df_RPT_summed.sort_values(by=['cell', 'RPT_No'])
df_RPT_summed

,RPT_No,cell,Charge_throughput [A.h]
0,0,A,63.819202
6,1,A,25.724848
12,2,A,35.011572
18,3,A,25.341377
24,4,A,34.421050
...,...,...,...
53,8,F,33.075788
59,9,F,23.985067
65,10,F,32.764889
71,11,F,23.702865


In [195]:
df_RPT_sorted.to_excel(Path_NiallThrCap+"Charge_throughput count - RPT.xlsx", index=False)
df_RPT_summed.to_excel(Path_NiallThrCap+"Charge_throughput sum - RPT.xlsx", index=False)

#### Now process the AGE 

In [160]:
all_files_age = os.listdir(Path_NiallThrCap + str_AGE)
txt_files = [file for file in all_files_age if file.endswith('.txt')]

In [161]:
def Read_Charge_Throuput_AGE(Output_dict,mpt_file,str_appx,str,str_saveFig,dpi): # str can be str_AGE or str_RPT
    # step-1: get the RPT name - input: filename
    degC_index = mpt_file.find('degC')
    mpt_index = mpt_file.find(str_appx)
    if degC_index != -1 and mpt_index != -1:
        extracted_part = mpt_file[degC_index -2  :  mpt_index]
        # print("Extracted Part:", extracted_part)
    else:
        print(f"Either 'degC' or {str_appx} not found in the filename.")
    # step-2: read data file
    colsToKeep = ["time/s", "dq/mA.h"]
    try:
        data = pd.read_csv(
            Path_NiallThrCap + str + "/" + mpt_file,
            encoding = "shift-jis", sep = "\t", usecols = colsToKeep)
    except:
        print(f"{mpt_file} doesn't have dq/mA.h, skip!")
        data = "Empty"
    else:
        # step-3: plot and save 
        data["dq/mA.h"] = abs(data["dq/mA.h"])
        fig, axs = plt.subplots(figsize=(6,5),tight_layout=True)
        axs.plot(data["time/s"]/3600,np.cumsum(data["dq/mA.h"])/1e3)
        axs.set_ylabel("Charge Throughput / A.h")
        axs.set_xlabel("Time / h")
        axs.set_title(extracted_part,fontsize=fs)
        plt.savefig(Path_NiallThrCap + str_saveFig + f"/Charge Throughput of {extracted_part}.png", dpi=dpi)
        plt.close()
        # step-4: get output dataframe
        Start_time = data["time/s"].iloc[0]/3600
        End_time   = data["time/s"].iloc[-1]/3600
        Charge_throughput = np.sum(abs(data["dq/mA.h"]))/1000 # in A.h
        print(extracted_part,Start_time,End_time,Charge_throughput)
        Output_dict["File_name"].append(extracted_part)
        Output_dict["Start_time [h]"].append(Start_time)
        Output_dict["End_time [h]"].append(End_time)
        Output_dict["Charge_throughput [A.h]"].append(Charge_throughput)
        print(f"Successfully process {extracted_part}")
    return Output_dict,data
# AGE_dict,data = Read_Charge_Throuput_AGE(Output_dict,txt_files[1],".txt",str_AGE,str_saveFig,dpi=300)

In [162]:
AGE_dict = {}
AGE_dict["File_name"] = []
AGE_dict["Start_time [h]"] = []
AGE_dict["End_time [h]"] = []
AGE_dict["Charge_throughput [A.h]"] = []
for txt_file in txt_files:
    AGE_dict,data = Read_Charge_Throuput_AGE(AGE_dict,txt_file,".txt",str_AGE,str_saveFig,dpi=300)

NDK - LG M50 deg - exp 2,2 - rig 6 - 40degC - cell F - cycling1(part2)_02_CP_CF5.txt doesn't have dq/mA.h, skip!
40degC - cell E - cycling4_CF4 0.0 332.5440938400364 759.3243768235795
Successfully process 40degC - cell E - cycling4_CF4
25degC - cell D - cycling10_CA5 0.0 335.2142430472297 759.8311841710296
Successfully process 25degC - cell D - cycling10_CA5
25degC - cell D - cycling11_CA5 0.0 281.0077450716161 638.0447850727844
Successfully process 25degC - cell D - cycling11_CA5
40degC - cell F - cycling8_CF5 0.0 334.0967668029222 760.0954746127153
Successfully process 40degC - cell F - cycling8_CF5
25degC - cell C - cycling8_CA4 0.0 334.05320235329003 759.4513397816617
Successfully process 25degC - cell C - cycling8_CA4
25degC - cell C - cycling12_CA4 0.0 333.91512344793 759.3239075163916
Successfully process 25degC - cell C - cycling12_CA4
40degC - cell F - cycling11_part2_CF5 0.0 57.941942915139556 123.40263224591381
Successfully process 40degC - cell F - cycling11_part2_CF5
10deg

In [163]:
df_AGE = pd.DataFrame(AGE_dict)
df_AGE

,File_name,Start_time [h],End_time [h],Charge_throughput [A.h]
0,40degC - cell E - cycling4_CF4,0.0,332.544094,759.324377
1,25degC - cell D - cycling10_CA5,0.0,335.214243,759.831184
2,25degC - cell D - cycling11_CA5,0.0,281.007745,638.044785
3,40degC - cell F - cycling8_CF5,0.0,334.096767,760.095475
4,25degC - cell C - cycling8_CA4,0.0,334.053202,759.451340
...,...,...,...,...
104,10degC - cell B - cycling1(part2)_01_MB_CB5,0.0,2.246156,1.482001
105,40degC - cell F - cycling1_CF5,0.0,23.934513,45.479969
106,25degC - cell D - cycling3_CA5,0.0,332.746873,757.578399
107,40degC - cell E - cycling11_part2_CF4,0.0,58.478870,124.210512


In [173]:
file_name = df_AGE["File_name"].iloc[10]
print(file_name)
def extract_values_age(file_name):
    index_c   = file_name.find("cell")
    index_cyc = file_name.find("cycling")
    Age = file_name[index_cyc+7:]
    if file_name[index_cyc+8] in [str(i) for i in range(10)]:
        Age_No = int(file_name[index_cyc+7:index_cyc+9])
    else:
        Age_No = int(file_name[index_cyc+7])
    T_age = file_name[0:2]
    str_cell = file_name[index_c + 5]
    return pd.Series({'Age': Age,'Age_No': Age_No, 'T_age': T_age, 'cell': str_cell})
# Apply the function to each row and create new columns
df_AGE[['Age','Age_No', 'T_age', 'cell']] = df_AGE['File_name'].apply(extract_values_age)

25degC - cell C - cycling9_CA4


In [182]:
df_AGE_sorted = df_AGE.sort_values(by=['cell', 'Age_No'])
df_AGE_sorted = df_AGE_sorted.reset_index(drop=True)
df_AGE_sorted

,File_name,Start_time [h],End_time [h],Charge_throughput [A.h],Age,T_age,cell,Age_No
0,10degC - cell A - cycling1(part2)_03_MB_CB4,311.4396,312.292850,1.178632,1(part2)_03_MB_CB4,10,A,1
1,10degC - cell A - cycling1_CB4,0.0000,23.943927,44.333110,1_CB4,10,A,1
2,10degC - cell A - cycling1(part2)_01_MB_CB4,0.0000,2.698156,1.917839,1(part2)_01_MB_CB4,10,A,1
3,10degC - cell A - cycling2_CB4,0.0000,333.974490,759.351559,2_CB4,10,A,2
4,10degC - cell A - cycling3_CB4,0.0000,333.226134,757.622541,3_CB4,10,A,3
...,...,...,...,...,...,...,...,...
104,40degC - cell F - cycling9_CF5,0.0000,333.944126,759.918594,9_CF5,40,F,9
105,40degC - cell F - cycling10_CF5,0.0000,334.903716,759.912783,10_CF5,40,F,10
106,40degC - cell F - cycling11_part2_CF5,0.0000,57.941943,123.402632,11_part2_CF5,40,F,11
107,40degC - cell F - cycling11_CF5,0.0000,281.064997,638.942860,11_CF5,40,F,11


In [192]:
df_AGE_summed =  df_AGE_sorted.groupby(['Age_No', 'cell'])['Charge_throughput [A.h]'].sum().reset_index()
df_AGE_summed =  df_AGE_summed.sort_values(by=['cell', 'Age_No'])
df_AGE_summed

In [194]:
df_AGE_sorted.to_excel(Path_NiallThrCap+"Charge_throughput count - AGE.xlsx", index=False)
df_AGE_summed.to_excel(Path_NiallThrCap+"Charge_throughput sum - AGE.xlsx", index=False)


#### Read for abnormal age ones

In [ ]:
# SKIP ONES:
# NDK - LG M50 deg - exp 2,2 - rig 3 - 25degC - cell C - cycling1(part2)_02_CP_CA4.txt doesn't have dq/mA.h, skip!
# NDK - LG M50 deg - exp 2,2 - rig 1 - 10degC - cell A - cycling1(part2)_02_CP_CB4.txt doesn't have dq/mA.h, skip!
# NDK - LG M50 deg - exp 2,2 - rig 6 - 40degC - cell E - cycling1(part2)_02_CP_CF4.txt doesn't have dq/mA.h, skip!
# NDK - LG M50 deg - exp 2,2 - rig 1 - 10degC - cell B - cycling1(part2)_02_CP_CB5.txt doesn't have dq/mA.h, skip!
# NDK - LG M50 deg - exp 2,2 - rig 3 - 25degC - cell D - cycling1(part2)_02_CP_CA5.txt doesn't have dq/mA.h, skip!
# NDK - LG M50 deg - exp 2,2 - rig 6 - 40degC - cell F - cycling1(part2)_02_CP_CF5.txt doesn't have dq/mA.h, skip!

In [99]:
txt_age_abnormal_files =[
    "NDK - LG M50 deg - exp 2,2 - rig 3 - 25degC - cell C - cycling1(part2)_02_CP_CA4.txt",
    "NDK - LG M50 deg - exp 2,2 - rig 1 - 10degC - cell A - cycling1(part2)_02_CP_CB4.txt",
    "NDK - LG M50 deg - exp 2,2 - rig 6 - 40degC - cell E - cycling1(part2)_02_CP_CF4.txt",
    "NDK - LG M50 deg - exp 2,2 - rig 1 - 10degC - cell B - cycling1(part2)_02_CP_CB5.txt",
    "NDK - LG M50 deg - exp 2,2 - rig 3 - 25degC - cell D - cycling1(part2)_02_CP_CA5.txt",
    "NDK - LG M50 deg - exp 2,2 - rig 6 - 40degC - cell F - cycling1(part2)_02_CP_CF5.txt",
]


In [151]:
def Read_Charge_Throuput_AGE_Abnormal(Output_dict,mpt_file,str_appx,str,str_saveFig,dpi): # str can be str_AGE or str_RPT
    # step-1: get the RPT name - input: filename
    degC_index = mpt_file.find('degC')
    mpt_index = mpt_file.find(str_appx)
    if degC_index != -1 and mpt_index != -1:
        extracted_part = mpt_file[degC_index -2  :  mpt_index]
        # print("Extracted Part:", extracted_part)
    else:
        print(f"Either 'degC' or {str_appx} not found in the filename.")
    # step-2: read data file
    try:
        data = pd.read_csv(
            Path_NiallThrCap + str + "/" + mpt_file,
            encoding = "shift-jis", sep = "\t")
    except:
        print(f"{mpt_file} doesn't have dq/mA.h, skip!")
        data = "Empty"
    else:
        fig, axs = plt.subplots(figsize=(8,5),tight_layout=True)
        diff_Q_dch= np.diff(data["Q discharge/mA.h"])/1e3
        diff_Q_cha= np.diff(data["Q charge/mA.h"])/1e3
        Q_tot = diff_Q_dch + diff_Q_cha
        Charge_throughput = np.cumsum(Q_tot)
        Ind_dch = np.where(diff_Q_dch < 0)[0]
        Ind_cha = np.where(diff_Q_cha < 0)[0]
        axs.plot(data["time/s"].iloc[0:-1][Ind_dch]/3600,  abs(diff_Q_dch[Ind_dch]),"-o",label="discharge")
        axs.plot(data["time/s"].iloc[0:-1][Ind_cha]/3600,  abs(diff_Q_cha[Ind_cha]),"-s",label="charge")
        axs.set_ylabel("Capacity for one step / A.h")
        axs.set_xlabel("Time / h")
        axs.legend()
        axs.set_title(extracted_part,fontsize=fs)
        plt.savefig(Path_NiallThrCap + str_saveFig + f"/Capacity for one step of {extracted_part}.png", dpi=dpi)
        plt.close()
        # step-4: get output dataframe
        Start_time = data["time/s"].iloc[0]/3600
        End_time   = data["time/s"].iloc[-1]/3600
        Charge_throughput = sum(abs(diff_Q_dch[Ind_dch])) +  sum(abs(diff_Q_cha[Ind_cha]))  # in A.h
        print(extracted_part,Start_time,End_time,Charge_throughput)
        Output_dict["File_name"].append(extracted_part)
        Output_dict["Start_time [h]"].append(Start_time)
        Output_dict["End_time [h]"].append(End_time)
        Output_dict["Charge_throughput [A.h]"].append(Charge_throughput)
        print(f"Successfully process {extracted_part}")
    return Output_dict,data,Ind_dch, Ind_cha


In [152]:
AGE_ab_dict,data,Ind_dch, Ind_cha = Read_Charge_Throuput_AGE_Abnormal(
        AGE_ab_dict,txt_age_abnormal_file,".txt",str_AGE,str_saveFig,dpi=400)

40degC - cell F - cycling1(part2)_02_CP_CF5 2.5830262337982677 310.69172290170667 710.4899597950109
Successfully process 40degC - cell F - cycling1(part2)_02_CP_CF5


In [154]:
print(len(Ind_dch),len(Ind_cha))

487 486


In [146]:
AGE_ab_dict = {}
AGE_ab_dict["File_name"] = []
AGE_ab_dict["Start_time [h]"] = []
AGE_ab_dict["End_time [h]"] = []
AGE_ab_dict["Charge_throughput [A.h]"] = []
for txt_age_abnormal_file in txt_age_abnormal_files:
    AGE_ab_dict,data = Read_Charge_Throuput_AGE_Abnormal(
        AGE_ab_dict,txt_age_abnormal_file,".txt",str_AGE,str_saveFig,dpi=400)

25degC - cell C - cycling1(part2)_02_CP_CA4 2.138572323798952 310.3648756405153 711.0060985765776
Successfully process 25degC - cell C - cycling1(part2)_02_CP_CA4
10degC - cell A - cycling1(part2)_02_CP_CB4 2.6982179059362457 311.43958132420255 711.8987488329414
Successfully process 10degC - cell A - cycling1(part2)_02_CP_CB4
40degC - cell E - cycling1(part2)_02_CP_CF4 2.660757348601409 310.7694540220842 710.5522850102841
Successfully process 40degC - cell E - cycling1(part2)_02_CP_CF4
10degC - cell B - cycling1(part2)_02_CP_CB5 2.2462162178006513 307.0344152435128 706.0597189332545
Successfully process 10degC - cell B - cycling1(part2)_02_CP_CB5
25degC - cell D - cycling1(part2)_02_CP_CA5 2.357981778664786 311.09934517251696 712.1584592256265
Successfully process 25degC - cell D - cycling1(part2)_02_CP_CA5
40degC - cell F - cycling1(part2)_02_CP_CF5 2.5830262337982677 310.69172290170667 710.4899597950109
Successfully process 40degC - cell F - cycling1(part2)_02_CP_CF5


In [147]:
df_AGE_ab = pd.DataFrame(AGE_ab_dict)
df_AGE_ab

,File_name,Start_time [h],End_time [h],Charge_throughput [A.h]
0,25degC - cell C - cycling1(part2)_02_CP_CA4,2.138572,310.364876,711.006099
1,10degC - cell A - cycling1(part2)_02_CP_CB4,2.698218,311.439581,711.898749
2,40degC - cell E - cycling1(part2)_02_CP_CF4,2.660757,310.769454,710.552285
3,10degC - cell B - cycling1(part2)_02_CP_CB5,2.246216,307.034415,706.059719
4,25degC - cell D - cycling1(part2)_02_CP_CA5,2.357982,311.099345,712.158459
5,40degC - cell F - cycling1(part2)_02_CP_CF5,2.583026,310.691723,710.489960


In [148]:
# Apply the function to each row and create new columns
df_AGE_ab[['Age', 'T_age', 'cell']] = df_AGE_ab['File_name'].apply(extract_values_age)
df_AGE_ab_sorted = df_AGE_ab.sort_values(by=['cell', 'Age'])
df_AGE_ab_sorted = df_AGE_ab_sorted.reset_index(drop=True)
df_AGE_ab_sorted

,File_name,Start_time [h],End_time [h],Charge_throughput [A.h],Age,T_age,cell
0,10degC - cell A - cycling1(part2)_02_CP_CB4,2.698218,311.439581,711.898749,1(part2)_02_CP_CB4,10,A
1,10degC - cell B - cycling1(part2)_02_CP_CB5,2.246216,307.034415,706.059719,1(part2)_02_CP_CB5,10,B
2,25degC - cell C - cycling1(part2)_02_CP_CA4,2.138572,310.364876,711.006099,1(part2)_02_CP_CA4,25,C
3,25degC - cell D - cycling1(part2)_02_CP_CA5,2.357982,311.099345,712.158459,1(part2)_02_CP_CA5,25,D
4,40degC - cell E - cycling1(part2)_02_CP_CF4,2.660757,310.769454,710.552285,1(part2)_02_CP_CF4,40,E
5,40degC - cell F - cycling1(part2)_02_CP_CF5,2.583026,310.691723,710.489960,1(part2)_02_CP_CF5,40,F


In [149]:
df_AGE_ab_sorted.to_excel(Path_NiallThrCap+"Charge_throughput count - AGE_ab.xlsx", index=False)